In [30]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import os


#mounting gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# import cv2

# def pad_image(image, target_size):
#     w, h = image.shape[:2]
#     top, bottom, left, right = 0, 0, 0, 0
#     if w < h:
#         diff = h - w
#         top = diff // 2
#         bottom = diff - top
#     else:
#         diff = w - h
#         left = diff // 2
#         right = diff - left
#     padding = (top, bottom, left, right)
#     return cv2.copyMakeBorder(image, *padding, cv2.BORDER_CONSTANT, (0, 0, 0))

#   padded_image = pad_image(image, (240, 240))


In [32]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from pathlib import Path


normal_images_path = Path("/content/drive/MyDrive/no")
tumor_images_path = Path("/content/drive/MyDrive/yes")


In [33]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True

)

validation_datagen = ImageDataGenerator(rescale=1./255)


In [34]:
img_height=240
img_width=240
batch_size=3

train_generator = train_datagen.flow_from_directory(
    directory=os.path.join(normal_images_path, "train"),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    # labels=["normal"]
)

validation_generator = validation_datagen.flow_from_directory(
    directory=os.path.join(normal_images_path, "val"),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    # labels=["normal"]
)

tumor_train_generator = train_datagen.flow_from_directory(
    directory=os.path.join(tumor_images_path, "train"),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    # labels=["tumor"]
)

tumor_validation_generator = validation_datagen.flow_from_directory(
    directory=os.path.join(tumor_images_path, "val"),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    # labels=["tumor"]
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [37]:
combined_train_generator = tf.keras.utils.Sequence(
    lambda: zip(train_generator, tumor_train_generator)
)

combined_validation_generator = tf.keras.utils.Sequence(
    lambda: zip(validation_generator, tumor_validation_generator)
)


TypeError: Sequence() takes no arguments

In [36]:
import tensorflow as tf

image_paths = tf.concat([tf.io.gfile.glob(os.path.join(tumor_images_path, "*")),
                      tf.io.gfile.glob(os.path.join(normal_images_path, "*"))], axis=0)

dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

# Optionally, pre-process your images (e.g., resizing) here
dataset = dataset.map(lambda path, label: (tf.image.decode_jpeg(tf.io.read_file(path)), label))

# Shuffle and batch your dataset for training
dataset = dataset.shuffle(buffer_size=len(labels)).batch(batch_size)


NameError: name 'labels' is not defined

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.MaxPooling2D((2, 2)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10)  # Adjust epochs as needed

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)